In [30]:
import pandas as pd

In [31]:
# slots.csv hora está armado a mano. Pero se podría armar a partir de las columnas del Form de respuestas
slots_info = "slots_dic_2024.csv"
df_slots = pd.read_csv(f"../datos/{slots_info}")
df_slots

,descripcion_form,dia,slot
0,Indique su disponibilidad horaria (hora de Arg...,Lunes,1
1,Indique su disponibilidad horaria (hora de Arg...,Lunes,3
2,Indique su disponibilidad horaria (hora de Arg...,Lunes,5
3,Indique su disponibilidad horaria (hora de Arg...,Lunes,7
4,Indique su disponibilidad horaria (hora de Arg...,Martes,1
5,Indique su disponibilidad horaria (hora de Arg...,Martes,3
6,Indique su disponibilidad horaria (hora de Arg...,Martes,5
7,Indique su disponibilidad horaria (hora de Arg...,Martes,7
8,Indique su disponibilidad horaria (hora de Arg...,Miércoles,1
9,Indique su disponibilidad horaria (hora de Arg...,Miércoles,3


In [32]:
# Preprocesa el archivo con las respuestas del form y genera un csv con el formato listo para el Tabu search

file='Encuesta de disponibilidad de días y horarios de los jurados - Diciembre 2024 (respuestas) - Respuestas de formulario 1.csv'
path=f"../datos/{file}"

df_form= pd.read_csv(path)
df_form.head()



,Marca temporal,Dirección de correo electrónico,N° de teléfono de contacto (exclusivamente para tener otra vía adicional de contacto durante las jornadas y solo en caso de emergencia),Seleccione su email,Indique su disponibilidad horaria (hora de Argentina) [Lunes 9 de diciembre - 16:00 a 17:30 h],Indique su disponibilidad horaria (hora de Argentina) [Lunes 9 de diciembre - 17:30 a 19:00 h],Indique su disponibilidad horaria (hora de Argentina) [Lunes 9 de diciembre - 19:00 a 20:30 h],Indique su disponibilidad horaria (hora de Argentina) [Lunes 9 de diciembre - 20:30 a 22:00 h],Indique su disponibilidad horaria (hora de Argentina) [Martes 10 de diciembre - 16:00 a 17:30 h],Indique su disponibilidad horaria (hora de Argentina) [Martes 10 de diciembre - 17:30 a 19:00 h],Indique su disponibilidad horaria (hora de Argentina) [Martes 10 de diciembre - 19:00 a 20:30 h],Indique su disponibilidad horaria (hora de Argentina) [Martes 10 de diciembre - 20:30 a 22:00 h],Indique su disponibilidad horaria (hora de Argentina) [Miércoles 11 de diciembre - 16:00 a 17:30 h],Indique su disponibilidad horaria (hora de Argentina) [Miércoles 11 de diciembre - 17:30 a 19:00 h],Indique su disponibilidad horaria (hora de Argentina) [Miércoles 11 de diciembre - 19:00 a 20:30 h],Indique su disponibilidad horaria (hora de Argentina) [Miércoles 11 de diciembre - 20:30 a 22:00 h],Indique su disponibilidad horaria (hora de Argentina) [Jueves 12 de diciembre - 16:00 a 17:30 h],Indique su disponibilidad horaria (hora de Argentina) [Jueves 12 de diciembre - 17:30 a 19:00 h],Indique su disponibilidad horaria (hora de Argentina) [Jueves 12 de diciembre - 19:00 a 20:30 h],Indique su disponibilidad horaria (hora de Argentina) [Jueves 12 de diciembre - 20:30 a 22:00 h]
0,24/11/2024 14:25:57,maxit1992@gmail.com,3412799120,maxit1992@gmail.com,No,"Sí, si fuera necesario",Sí,Sí,No,"Sí, si fuera necesario",Sí,Sí,No,"Sí, si fuera necesario",Sí,Sí,No,"Sí, si fuera necesario",Sí,Sí
1,24/11/2024 15:25:22,dessaya@fi.uba.ar,11-6268-6792,dessaya@gmail.com,No,No,No,No,Sí,Sí,Sí,"Sí, si fuera necesario",Sí,Sí,Sí,"Sí, si fuera necesario",Sí,Sí,Sí,"Sí, si fuera necesario"
2,24/11/2024 16:27:56,cyanez@fi.uba.ar,1127141100,cyanez@fi.uba.ar,Sí,Sí,Sí,Sí,Sí,Sí,Sí,Sí,Sí,Sí,Sí,Sí,Sí,Sí,Sí,Sí
3,24/11/2024 18:04:13,vacagonzalo@gmail.com,1123681920,vacagonzalo@gmail.com,No,No,No,No,No,No,Sí,Sí,No,No,No,No,No,No,Sí,Sí
4,24/11/2024 19:06:25,avirgillo.95@gmail.com,1136354531,avirgillo.95@gmail.com,No,No,Sí,Sí,No,No,Sí,Sí,No,No,"Sí, si fuera necesario","Sí, si fuera necesario",No,No,Sí,Sí


In [33]:

# pivot de columnas de respuestas
cols_disponibilidad = [c for c in df_form.columns if c.startswith("Indique su disponibilidad")]
df_long = df_form.melt(
    id_vars=[c for c in df_form.columns if c not in cols_disponibilidad],
    value_vars=cols_disponibilidad,
    var_name='descripcion_form',
    value_name='respuesta'
)
#  merge con el csv de slots
df_merged = df_long.merge(df_slots, on='descripcion_form', how='left')

df_final = df_merged[['Seleccione su email', 'dia', 'slot', 'respuesta']].copy()
df_final.rename(columns={'Seleccione su email': 'jurado'}, inplace=True)
df_final['slot'] = df_final['slot'].astype(int)

map_respuesta = {'No': 0,'Sí, si fuera necesario': 1, 'Sí': 2}
df_final['disponibilidad'] = df_final['respuesta'].map(map_respuesta)

df_final.head()


,jurado,dia,slot,respuesta,disponibilidad
0,maxit1992@gmail.com,Lunes,1,No,0
1,dessaya@gmail.com,Lunes,1,No,0
2,cyanez@fi.uba.ar,Lunes,1,Sí,2
3,vacagonzalo@gmail.com,Lunes,1,No,0
4,avirgillo.95@gmail.com,Lunes,1,No,0


## Solo si quiero dividir los bloques de hora y media en dos bloques de 45 minutos

In [34]:
# Duplico la disponibilidad para que cada bloque de 1 hora y media aparezca como dos medios bloques de 45 minutos
df_dup = df_final.copy()
df_dup['slot'] = df_dup['slot'] + 1
df_extended = pd.concat([df_final, df_dup], ignore_index=True)
df_extended = df_extended.sort_values(by=['jurado', 'dia', 'slot']).reset_index(drop=True)
df_extended.head(10)

,jurado,dia,slot,respuesta,disponibilidad
0,Jose.Alamos@haw-hamburg.de,Jueves,1,Sí,2
1,Jose.Alamos@haw-hamburg.de,Jueves,2,Sí,2
2,Jose.Alamos@haw-hamburg.de,Jueves,3,Sí,2
3,Jose.Alamos@haw-hamburg.de,Jueves,4,Sí,2
4,Jose.Alamos@haw-hamburg.de,Jueves,5,"Sí, si fuera necesario",1
5,Jose.Alamos@haw-hamburg.de,Jueves,6,"Sí, si fuera necesario",1
6,Jose.Alamos@haw-hamburg.de,Jueves,7,No,0
7,Jose.Alamos@haw-hamburg.de,Jueves,8,No,0
8,Jose.Alamos@haw-hamburg.de,Lunes,1,Sí,2
9,Jose.Alamos@haw-hamburg.de,Lunes,2,Sí,2


In [35]:
df_extended.to_csv('../datos/disponibilidad_dic_2024.csv', index=False)

In [36]:
# split de los slots de 1 hora y media en slots de 45 minutos
from datetime import datetime, timedelta

nuevas_filas = []

for _, row in df_slots.iterrows():
    descripcion = row['descripcion_form']
    dia = row['dia']
    slot = row['slot']

    # Extraer la hora de inicio y fin de la descripción
    # Ej: "Indique su disponibilidad horaria ... [Lunes 20 de octubre - 16:00 a 17:30 h]"
    hora_str = descripcion.split('-')[-1].strip().replace('h]', '').strip()  # "16:00 a 17:30"
    inicio_str, fin_str = hora_str.split('a')
    inicio = datetime.strptime(inicio_str.strip(), "%H:%M")
    fin = datetime.strptime(fin_str.strip(), "%H:%M")

    # Calcular la mitad del slot (45 min)
    duracion = (fin - inicio) / 2
    mitad1_fin = inicio + duracion

    # Crear dos descripciones nuevas
    desc1 = descripcion.replace(f"{inicio.strftime('%H:%M')} a {fin.strftime('%H:%M')}", 
                                 f"{inicio.strftime('%H:%M')} a {mitad1_fin.strftime('%H:%M')}")
    desc2 = descripcion.replace(f"{inicio.strftime('%H:%M')} a {fin.strftime('%H:%M')}", 
                                 f"{mitad1_fin.strftime('%H:%M')} a {fin.strftime('%H:%M')}")

    # Añadir dos filas nuevas
    nuevas_filas.append({
        'descripcion_form': desc1, 
        'dia': dia, 
        'rango_horario': f"{inicio.strftime('%H:%M')} a {mitad1_fin.strftime('%H:%M')}",
        'slot': slot
    })
    nuevas_filas.append({
        'descripcion_form': desc2, 
        'dia': dia, 
        'rango_horario': f"{mitad1_fin.strftime('%H:%M')} a {fin.strftime('%H:%M')}",
        'slot': slot + 1
    })

df_nuevo = pd.DataFrame(nuevas_filas)
df_nuevo.head(10)

,descripcion_form,dia,rango_horario,slot
0,Indique su disponibilidad horaria (hora de Arg...,Lunes,16:00 a 16:45,1
1,Indique su disponibilidad horaria (hora de Arg...,Lunes,16:45 a 17:30,2
2,Indique su disponibilidad horaria (hora de Arg...,Lunes,17:30 a 18:15,3
3,Indique su disponibilidad horaria (hora de Arg...,Lunes,18:15 a 19:00,4
4,Indique su disponibilidad horaria (hora de Arg...,Lunes,19:00 a 19:45,5
5,Indique su disponibilidad horaria (hora de Arg...,Lunes,19:45 a 20:30,6
6,Indique su disponibilidad horaria (hora de Arg...,Lunes,20:30 a 21:15,7
7,Indique su disponibilidad horaria (hora de Arg...,Lunes,21:15 a 22:00,8
8,Indique su disponibilidad horaria (hora de Arg...,Martes,16:00 a 16:45,1
9,Indique su disponibilidad horaria (hora de Arg...,Martes,16:45 a 17:30,2


In [37]:
df_nuevo.to_csv("../datos/slots_divididos_dic_2024.csv", index=False)